In [72]:
import pandas as pd
import numpy as np
from datetime import datetime
import xlrd

courseList = {"CS 302": "Computer Science",
        "RHE 306": "ENGL 1301",
        "RHE 309K": "ENGL 1302",
        "GEO 302E": "Geoscience",
        "HIS 315K": "HIST 1301",
        "HIS 315L": "HIST 1302",
        "M 305G": "Precalculus",
        "PHY 302K": "PHYS 1301",
        "PHY 302L": "PHYS 1302",
        "PHY 1403": "PHY 1403",
        "SDS 302": "Statistics",
        "AET 304": "AET",
        "CH 301": "Chemistry",
        "CH 104M": "Chemistry Lab",
        "M 301": "College Algebra",
        "NSC 309": "NSC 309"}
courseList = pd.Series(courseList)

def remove(data):
    removal = []
    for i in removal:
        data = data[data["Course Code"] != i]
    return data
    
# Naming convention to be updated
def standardize(data):
    convention = courseList
    for i in convention.index:
        data.loc[data["Course Code"] == i, "Course Title"] = convention[i]
    return data

#Load Data
def get_master(master, district = ''):
    data = pd.read_csv(master)
    data = remove(data)
    data = data[data["Eligibility Status Final Simple"] != 'HS Only']
    data = standardize(data)
    if district == '':
        return data
    return data[data["District Name"] == district]

def read_data(master, district):
    data = get_master(master, district)
    UT = data[data["Partner"] == "UT Austin OnRamps"]
    TTU = data[data["Partner"] == "TTU OnRamps"]
    return UT, TTU

def read_data_without_differentiate(master, district):
    data = get_master(master, district)
    return data

# Get Row totals
def get_row_total(data2, clean = True):
    if clean:
        data = pd.DataFrame(columns = data2.columns)
        data['Total'] = 0
        for index, row in data2.iterrows():
            rowSum = sum(data2.loc[index])
            if rowSum > 0:
                data.loc[index] = data2.loc[index]
                data.loc[index, 'Total'] = rowSum
    else:
        data = data2   
        data['Total'] = 0
        for index, row in data.iterrows():
            data.loc[index, 'Total'] = sum(data.loc[index])
    return data

# Get Column totals
def get_column_total(data2, clean = True):
    if clean:
        data = pd.DataFrame(index = data2.index)
        for i in data2.columns:
            colSum = data2[i].sum()
            if colSum > 0:
                data[i] = data2[i]
                data.loc['Total', i] = colSum
    else:
        data = data2
        data.loc['Total'] = 0
        for i in data.columns:
            data.loc['Total', i] = data[i].sum()
    return data
    
def get_Campus(data):
    return data['Campus Name'].drop_duplicates()

def get_unique_values(date, label):
    return data[label].drop_duplicates()

def get_percent(data, total = 0, column = '', column2 = None):
    order = []
    for i in data.columns:
        order.append(i)
        if total != 0:
            order.append(i + '_%')
            data[i + '_%'] = data[i]/total
        elif column != '':
            order.append(i + '_%')
            data[i + '_%'] = data[i]/data['Total'] 
        elif column2 is not None:
            order.append(i + '_%')
            data[i + '_%'] = data[i]/column2
    return data[order]

def get_course(data, label = []):
    if label == '':
        course = data["Course Code"]
    else:
        course = data[label]
    return course.drop_duplicates()

## See EOY Figure 3_2 Similar Pattern
def Figure_3_2(data, district):
    data = data[data["Eligibility Status Final Simple"] == "Eligible"]
    grade = ['Yes', 'No']
    ans = pd.DataFrame(index = pd.Index(courseList), columns = grade)
    for i in grade:
        ans[i] = data[data["College Credit Earned Final"] == i]["Course Title"].value_counts()
    ans = ans.fillna(0)
    ans.columns = ['Credit Earned', 'Credit Not Earned']
    ans = ans.sort_index()
    return ans

## See MOY Figure 5_3 Similar pattern
def Figure_5_3(data, district):
    past = data[0]
    current = data[1]
    past = past[past["District Name"] == district]
    current = current[current["District Name"] == district]
    past = Figure_3_2(past, district)
    past = get_column_total(past)
    past = get_row_total(past)
    past = get_percent(past, column2 = past['Total'])
    past.columns = past.columns + '_' + '1718'
    current = Figure_3_2(current, district)
    current = get_column_total(current)
    current = get_row_total(current)
    current = get_percent(current, column2 = current['Total'])
    current.columns = current.columns + '_' + '1819'
    data = pd.concat([current, past], axis = 1, sort=False)
    data = data.sort_index()
    cols = []
    for i in ['Credit Earned', 'Credit Not Earned']:
        cols.append(i + '_1718')
        cols.append(i + '_%_1718')
        cols.append(i + '_1819')
        cols.append(i + '_%_1819')
    cols.append('Total_1718')
    cols.append('Total_1819')
    ans = pd.DataFrame(index = data.index, columns = cols)
    for i in ans.columns:
        if i in data.columns:
            ans[i] = data[i]
    ans['Total_%'] = 1
    ans = ans.fillna(0)
    return ans

def Figure_6_1(data, district):
    past = data[0]
    current = data[1]
    past = past[past["District Name"] == district]
    current = current[current["District Name"] == district]
    ans = Figure_5_3([past, current], district)
    course17 = past["Course Title"].drop_duplicates().tolist()
    course18 = current["Course Title"].drop_duplicates().tolist()
    template = []
    for i in ans.index:
        lrow = ['', '', i]
        rowData = ans.loc[i].tolist()
        if i != 'Total':
            if i not in course17:
                rowData[1] = '-'
                rowData[5] = '-'
            if i not in course18:
                rowData[3] = '-'
                rowData[7] = '-'
        lrow = lrow + rowData
        template.append(lrow)
    return template

def Figure_6_1_QC(data, district_name):
    
    template = [['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''],
['', '', '6.0 ANNUAL COMPARISON', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''],
['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''],
['', '', '', 'Figure 6.1 shows the percentage of eligible students who earned credit from the distance college course in 2017-2018 and 2018-2019.', '', '', '', '', '', '', '', '', '', '', '', '', '', ''],
['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''],
['', f'Figure 6.1: Student Eligibility Status by College Course in {district_name} in the 2016-2017 and 2017-2018 Academic Years', None, None, None, None, None, None, None, None, None, None, None, None, None, '', '', ''],
['', '', None, None, 'Credit Earned', None, None, None, 'Credit Not Earned', None, None, None, 'Total', None, None, '', '', ''],
['', 'Distance College Course', '', None, '17-18', None, '18-19', None, '17-18', None, '18-19', None, '17-18', '18-19', '17-18/18-19', '', '', ''],
['', '', None, None, 'N', '%', 'N', '%', 'N', '%', 'N', '%', 'N', 'N', '%', '', '', '']
        ]
    
    template = template + data
    return template

def filter_str(row):
    if row is None:
        return row
    newRow = []
    for j in row:
        if isinstance(j, float):
            newRow.append(round(j, 10))
        elif j != '' and j != None:
            newRow.append(j)
    return newRow

def verify(inputFile, QCData, index):
    loc = (inputFile)
    wb = xlrd.open_workbook(loc) 
    sheet = wb.sheet_by_index(index)
    QCCompleted = 1
    for i in range(sheet.nrows):
        sheetRow = filter_str(sheet.row_values(i))
        QCRow = filter_str(QCData[i])
        if sheetRow != QCRow:
            print(inputFile + " (sheet " + str(index + 1) + ") line " + str(i) + " Error!")
            print("EOY Output:")
            print(sheetRow)
            print("QC Output:")
            print(QCRow)
            QCCompleted = 0
    return True

In [73]:
def report_generator_UT_TTU(master, master1718, district):
    
    OnRamps = standardize(read_data_without_differentiate(master, district))
    UT, TTU = read_data(master, district)
    UT17, TTU17 = read_data(master1718, district)

    funclist = [([UT17, UT], Figure_6_1, Figure_6_1_QC)]
    for i in funclist:
        data, func, QC= i
        return verify(district + ".xlsx", QC(func(data, district), district), 16)

master = "18-19 FAKE Master File (Final).csv"
master1718 = '17-18 Master File (Student Tab)_Final (2018-05-29).csv'
district = pd.read_csv("DistrictName2.csv", header = None)

for i in range(len(district)):
    distrct_name = district.loc[i, 0]
    try:
        if report_generator_UT_TTU(master, master1718, distrct_name) == True:
            if i < len(district) - 1:
                district.loc[i + 1 :].to_csv("district not finished.csv", index=False)
    except FileNotFoundError as error:
        print(distrct_name)
        print(error)

Austin ISD.xlsx (sheet 17) line 9 Error!
EOY Output:
['AET', 0.0, 0.0, 25.0, 0.5, 0.0, 0.0, 25.0, 0.5, 0.0, 50.0, 1.0]
QC Output:
['AET', 0.0, '-', 21.0, 1.0, 0.0, '-', 0.0, 0.0, 0.0, 21.0, 1.0]
Austin ISD.xlsx (sheet 17) line 10 Error!
EOY Output:
['Chemistry Lab', 0.0, 0.0, 10.0, 0.5, 0.0, 0.0, 10.0, 0.5, 0.0, 20.0, 1.0]
QC Output:
['Chemistry', 0.0, '-', 14.0, 1.0, 0.0, '-', 0.0, 0.0, 0.0, 14.0, 1.0]
Austin ISD.xlsx (sheet 17) line 11 Error!
EOY Output:
['Chemistry', 0.0, 0.0, 18.0, 0.5, 0.0, 0.0, 18.0, 0.5, 0.0, 36.0, 1.0]
QC Output:
['Chemistry Lab', 0.0, '-', 7.0, 0.875, 0.0, '-', 1.0, 0.125, 0.0, 8.0, 1.0]
Austin ISD.xlsx (sheet 17) line 12 Error!
EOY Output:
['College Algebra', 0.0, 0.0, 98.0, 0.5, 0.0, 0.0, 98.0, 0.5, 0.0, 196.0, 1.0]
QC Output:
['College Algebra', 0.0, '-', 80.0, 0.824742268, 0.0, '-', 17.0, 0.175257732, 0.0, 97.0, 1.0]
Austin ISD.xlsx (sheet 17) line 13 Error!
EOY Output:
['Computer Science', 39.0, 0.7959183673, 51.0, 0.5, 10.0, 0.2040816327, 51.0, 0.5, 49.0

In [ ]:
loc = ("Austin ISD.xlsx") 
wb = xlrd.open_workbook(loc) 
sheet = wb.sheet_by_index(16) 
for i in range(sheet.nrows):
    print(sheet.row_values(i))